In [ ]:
import numpy as np
import cv2
from keras.models import load_model
face_cascade = cv2.CascadeClassifier("C:\Users\saura\OneDrive\Desktop\Interative Bot\Emotion detection\Emotion detection_old\haarcascade_frontalface_default.xml")
model = load_model("C:\Users\saura\OneDrive\Desktop\Interative Bot\Emotion detection\Emotion detection_old\fer2013_mini_XCEPTION.102-0.66.hdf5")


labels = {0:'angry',1:'disgust',2:'fear',3:'happy',
                4:'sad',5:'surprise',6:'neutral'}
cap = cv2.VideoCapture(0)
while True:
    imagevalid, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for face in faces:
        x, y, w, h = face
        face2feed = gray[y:y+h, x:x+w]
        face2feed = cv2.resize(face2feed, tuple(model.input_shape[1:3]))
        face2feed = face2feed.astype('float32')
        face2feed = face2feed / 255.0
        face2feed = face2feed - 0.5
        face2feed = face2feed * 2.0
        face2feed = np.array(face2feed)
        face2feed = face2feed.reshape(1, 64, 64, 1)
        emotionscores = model.predict(face2feed)
        emotionprob = np.max(emotionscores)
        emotionindx = np.argmax(emotionscores)
        emotion = labels[emotionindx]
        if emotion == 'angry':
            color = emotionprob * np.array((0, 0, 255))
        elif emotion == 'sad':
            color = emotionprob * np.array((255, 0, 0))
        elif emotion == 'happy':
            color = emotionprob * np.array((255, 255, 0))
        elif emotion == 'surprise':
            color = emotionprob * np.array((0, 255, 255))
        else:
            color = emotionprob * np.array((0, 255, 0))
        cv2.rectangle(image, (x,y), (x+w,y+h), tuple(color), 2)
        cv2.putText(image, emotion, (x, y),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, color, 2, cv2.LINE_AA)
    cv2.imshow('Emotion predicted', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()